# Description - Table 02 SI using tvFC in original space

This notebook will compute the Silhouette Index when using as input tvFC matrices without any dimensionality reduction.

* SItask at the scan level --> one value per subject --> we report mean and stdev

* SIsbj at the group level --> only one value available

We do this using as input both the normalized and non-normalized matrices

In [1]:
import argparse
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score
from utils.basics import PRJ_DIR, PNAS2015_subject_list
import os.path as osp
from tqdm import tqdm

In [2]:
norm_methods = ['asis','zscored']

# 1. Scan level embeddings (SI task)

In [3]:
%%time
df = pd.DataFrame(columns=['Subject','Norm Method','SI'])
for sbj in tqdm(PNAS2015_subject_list, desc='Subject'):
    for norm_method in norm_methods:
        input_path      = osp.join(PRJ_DIR,'Data_Interim','PNAS2015',sbj,'Original',f'{sbj}_Craddock_0200.WL045s.WS1.5s.tvFC.Z.{norm_method}.pkl')
        data            = pd.read_pickle(input_path).T
        data.index.name = 'Task'
        data            = data.drop('XXXX') 
        input_data      = data.values
        input_labels    = list(data.index)
        si_value        = silhouette_score(input_data, input_labels, n_jobs=-1)
        df              = df.append({'Subject':sbj,'Norm Method':norm_method,'SI':si_value}, ignore_index=True)

Subject: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:55<00:00,  2.79s/it]

CPU times: user 16min 30s, sys: 31min 23s, total: 47min 53s
Wall time: 55.9 s


Here, we print the mean SI across all subjects.

In [4]:
df.groupby(by='Norm Method').mean().round(2)

,SI
Norm Method,
asis,0.13
zscored,0.12


Next, we print the standard deviation across all subjects

In [5]:
df.groupby(by='Norm Method').std().round(2)

,SI
Norm Method,
asis,0.02
zscored,0.01


# 2. Group Level SI

## 2.1. Load tvFC matrices for all subjects and concatenate them on a single large tvFC group matrix

In [6]:
%%time
# Create empty dictionary that will hold the group level tvFC matrices
group_data = {'asis':None,
              'zscored':None}

for norm_method in norm_methods:
    for sbj in tqdm(PNAS2015_subject_list, desc='Subject'):
        input_path      = osp.join(PRJ_DIR,'Data_Interim','PNAS2015',sbj,'Original',f'{sbj}_Craddock_0200.WL045s.WS1.5s.tvFC.Z.{norm_method}.pkl')
        data            = pd.read_pickle(input_path).T
        data.index.name = 'Task'
        data            = data.drop('XXXX') 
        data            = data.reset_index()
        data['Subject'] = sbj
        data            = data.set_index(['Subject','Task'])
        if group_data[norm_method] is None:
            group_data[norm_method] = data
        else:
            group_data[norm_method] = pd.concat([group_data[norm_method], data],axis=0)

Subject: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:12<00:00,  1.65it/s]

CPU times: user 13.6 s, sys: 10.3 s, total: 23.9 s
Wall time: 26.2 s


## 2.2. Compute SI using the group level tvFC matrices

In [7]:
%%time
for norm_method in norm_methods:
    print(f'++ INFO: Results for {norm_method} matrices')
    input_labels_sbj = list(group_data[norm_method].index.get_level_values('Subject'))
    input_labels_tsk = list(group_data[norm_method].index.get_level_values('Task'))
    input_data       = group_data[norm_method].values
    si_value_sbj     = silhouette_score(input_data, input_labels_sbj,n_jobs=-1)
    si_value_tsk     = silhouette_score(input_data, input_labels_tsk,n_jobs=-1)
    print('++ INFO: Group Level SItask    [%s] = %.02f' % (norm_method,si_value_tsk))
    print('++ INFO: Group Level SIsubject [%s] = %.02f' % (norm_method,si_value_sbj))

++ INFO: Results for asis matrices
++ INFO: Group Level SItask    [asis] = 0.01
++ INFO: Group Level SIsubject [asis] = 0.05
++ INFO: Results for zscored matrices
++ INFO: Group Level SItask    [zscored] = 0.01
++ INFO: Group Level SIsubject [zscored] = -0.02
CPU times: user 38min 27s, sys: 7min 44s, total: 46min 12s
Wall time: 53.1 s
